In [45]:
from pathlib import Path
from datetime import datetime

import gymnasium as gym
from gymnasium.wrappers import RecordVideo

from stable_baselines3 import A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [46]:
docs_path = Path("../../documentation/frozenlake/a2c-frozenlake") 

run_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_path = docs_path / f"run_{run_id}"

video_dir = run_path / "videos"
graphs_dir  = run_path / "graphs"
report_file = run_path / "run_log.md"
model_dir = run_path / "model"
monitor_dir = run_path / "monitor"

docs_path.mkdir(parents=True, exist_ok=True)
graphs_dir.mkdir(parents=True, exist_ok=True)
video_dir.mkdir(parents=True, exist_ok=True)
model_dir.mkdir(parents=True, exist_ok=True)
monitor_dir.mkdir(parents=True, exist_ok=True)

model_file = model_dir / "a2c_frozenlake"
monitor_file = str(monitor_dir)

In [47]:
env = gym.make("FrozenLake-v1", is_slippery=True, render_mode="rgb_array")
env = Monitor(env, monitor_file)

In [48]:
model = A2C(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=0.0005,
    gamma=0.99
) 
# verbose shows info printed while training
# learning rate how fast the neural network updates after each step ( small -> slow, large -> fast)
# gamma discount factor for rewards ( small-> focus on immediate rewards, large -> long term rewards)

model.learn(total_timesteps=100000)
model.save(str(model_file))

Using cpu device
Wrapping the env in a DummyVecEnv.
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.14      |
|    ep_rew_mean        | 0.0143    |
| time/                 |           |
|    fps                | 2788      |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.38     |
|    explained_variance | -1.47e+03 |
|    learning_rate      | 0.0005    |
|    n_updates          | 99        |
|    policy_loss        | -0.000361 |
|    value_loss         | 1.36e-07  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 7.44      |
|    ep_rew_mean        | 0         |
| time/                 |           |
|    fps                | 2886      |
|    iterations         | 200       |
|    time_elapsed       | 0         

In [49]:
# Reset environment
SEED = 42
observation, info = env.reset(seed=SEED)
print(f"Total Timesteps:{model.num_timesteps}")
print(f"Action space: {env.action_space}") 
print(f"Observation space: {env.observation_space}") 
print(f"Starting observation: {observation}")
print(f"maxiumum number of steps per episode: {env.spec.max_episode_steps}")

Total Timesteps:100000
Action space: Discrete(4)
Observation space: Discrete(16)
Starting observation: 0
maxiumum number of steps per episode: 100


In [50]:
step = 0
total_reward = 0
episode_over = False

In [51]:
labels = ["Left", "Down", "Right", "Up"]

while not episode_over:
    step += 1
    action, state = model.predict(observation,deterministic=True)  
    observation, reward, terminated, truncated, info = env.step(int(action))

    total_reward += reward
    episode_over = terminated or truncated

    print(f"Step {step}:")
    print(f"Action taken: {action}({labels[action]})")
    print(f"Reward: {reward}")
    print(f"Terminated: {terminated}, Truncated: {truncated}")
    print("-" * 50)

print(f"Episode finished! Total reward: {total_reward}")
env.close()

Step 1:
Action taken: 0(Left)
Reward: 0.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 2:
Action taken: 0(Left)
Reward: 0.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 3:
Action taken: 0(Left)
Reward: 0.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 4:
Action taken: 3(Up)
Reward: 0.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 5:
Action taken: 1(Down)
Reward: 0.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 6:
Action taken: 0(Left)
Reward: 0.0
Terminated: False, Truncated: False
--------------------------------------------------
Step 7:
Action taken: 1(Down)
Reward: 1.0
Terminated: True, Truncated: False
--------------------------------------------------
Episode finished! Total reward: 1.0


In [52]:
lake_layout = ["".join(row.decode("utf-8") for row in line) for line in env.unwrapped.desc]

obs_explanation = """\
**Observation (Discrete integer):**
- A single integer (0–15) representing the agent’s position on the 4×4 grid (16).
- Each cell in the grid corresponds to a unique state:
  - `S` — Start position (state 0)
  - `F` — Frozen tile (safe)
  - `H` — Hole (episode ends with no reward)
  - `G` — Goal (episode ends with +1 reward)

**Action Space (Discrete 4):**
- 0 → Left
- 1 → Down
- 2 → Right
- 3 → Up
"""

failure_conditions = """\
**Episode ends when (termination/truncation):**
- The agent **falls into a hole (`H`)** → `terminated = True`
- The agent **reaches the goal (`G`)** → `terminated = True`
- The **maximum number of steps (100)** is reached → `truncated = True`

**Reward Structure:**
- `+1` for reaching the goal (`G`)
- `0` for all other tiles (including holes)
"""


with open(report_file, "w",encoding="utf-8")as f:
    f.write("## Environment Details\n")
    f.write(f"- Action space: {env.action_space}\n")
    f.write(f"- Observation space: {env.observation_space}\n")
    f.write(f"- Maximum steps per episode: {env.spec.max_episode_steps}\n\n")

    f.write("## Frozen Lake Layout\n")
    for line in lake_layout:
        f.write(line + "\n")
    f.write("\n")


    f.write("## Observation Meaning\n")
    f.write(obs_explanation + "\n")

    f.write("## Failure Conditions\n")
    f.write(failure_conditions + "\n")


    f.write("## Performance Summary:\n")
    f.write(f"- Seed: {SEED}\n")
    f.write(f"- Total Timesteps: {model.num_timesteps}\n")
    f.write(f"- Algorithm: {type(model).__name__}\n")
    f.write(f"- Observation: {env.observation_space}\n")
    f.write(f"- Total reward: {total_reward}\n\n")

    f.write("## Graph Interpretation:\n")
    f.write("- The training curve visualizes total episode rewards across timesteps.\n")
    f.write("- The evaluation curve measures average policy performance at fixed intervals.\n")
    f.write("- A positive slope suggests the A2C agent is learning an effective navigation strategy.\n\n")


In [53]:
def generate_evaluation_curve(model, graphs_dir, num_eval_episodes=20, eval_freq=5000, total_timesteps=100000):
    """
    Evaluate the model at regular intervals and plot evaluation curve.
    Evaluation curve, not recording rewards during training, evaluating trained model at fixed intervals
    Each point on graph comes from evaluate_policy(), which tests model for several episodes & averages results
    Measures performance after certain amounts of training, NOT the actual learning process

    * TLDR: graphs shows the evaluarion reward at regular timesteps during training *
    """
    rewards = []
    timesteps = []

    env = gym.make("FrozenLake-v1", is_slippery=True)

    for step in range(eval_freq, total_timesteps + 1, eval_freq):
        # Evaluate policy
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=num_eval_episodes, deterministic=True) #this uses the model we just trained 
        rewards.append(mean_reward)
        timesteps.append(step)

    # Plot
    plt.figure(figsize=(8, 5))
    plt.plot(timesteps, rewards, marker="o", label="Mean Reward (Evaluation)", color="tab:orange")
    plt.xlabel("Timesteps")
    plt.ylabel("Mean Reward over Episodes")
    plt.title("evaluation Curve (A2C on frozenlake)")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend()

    # Save figure
    graph_file = graphs_dir / "evaluation_curve.png"
    plt.savefig(graph_file, dpi=200, bbox_inches="tight")
    plt.close()
    env.close()

    print(f"evaluation curve saved to {graph_file}")


generate_evaluation_curve(model, graphs_dir)


/Users/davidjayakumar/Library/CloudStorage/OneDrive-AtlanticTU/YEAR 4/Project Engineering/Network-Defender/.venv_mac/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


evaluation curve saved to ../../documentation/frozenlake/a2c-frozenlake/run_2025-10-25_14-45-31/graphs/evaluation_curve.png


In [54]:
def plot_training_curve(monitor_dir, graphs_dir):
    """
    Plot the training learning curve based on rewards logged during training.
    The learning curve visualizes how the agent’s performance evolves over time
    by showing the total episode rewards collected during training.

    Each point on the graph corresponds to the cumulative reward obtained in a
    single training episode, as recorded automatically by the Monitor wrapper.

    This curve reflects the agent’s actual *learning process* — showing how it
    improves, stabilizes, or fluctuates in performance while training.

    * TLDR: graph shows the agent’s reward per episode DURING training (true learning progress) *
  
    """
    monitor_dir = Path(monitor_dir)

    csv_files = list(monitor_dir.glob("*.csv"))
    if not csv_files:
        raise FileNotFoundError(f"No monitor CSV found in {monitor_dir}")
    log_file = csv_files[0]  

    print(f"Found monitor file: {log_file.name}")

    df = pd.read_csv(log_file, skiprows=1)
    plt.figure(figsize=(10, 6))
    plt.plot(df["r"], label="Reward per Episode", color="tab:blue", alpha=0.7)
    plt.xlabel("Episodes")
    plt.ylabel("Total Reward")
    plt.title("Training Learning Curve (A2C on FrozenLake)")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend()
    out = Path(graphs_dir) / "training_learning_curve.png"
    plt.savefig(out, dpi=200, bbox_inches="tight")
    plt.close()
    print(f"Saved training learning curve → {out}")

plot_training_curve(monitor_file, graphs_dir)


Found monitor file: monitor.csv
Saved training learning curve → ../../documentation/frozenlake/a2c-frozenlake/run_2025-10-25_14-45-31/graphs/training_learning_curve.png


In [55]:
def record_demo_videos(model_file, video_dir, num_episodes=5):
    """
    Record demo videos of the A2C agent before and after training.
    - 'Before training' shows a freshly initialized A2C agent with random behavior.
    - 'After training' shows the saved trained model's performance.
    """
    # --- Baseline (Before Training) ---
    print("\nRecording baseline A2C agent episodes...")
    env = gym.make("FrozenLake-v1", is_slippery=True, render_mode="rgb_array")
    baseline_model = A2C("MlpPolicy", env, verbose=0)

    baseline_env = RecordVideo(
        env,
        video_folder=str(video_dir),
        episode_trigger=lambda e: True,
        name_prefix="a2c_baseline_untrained"
    )

    for ep in range(num_episodes):
        obs, info = baseline_env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = baseline_model.predict(obs, deterministic=False)
            action = int(action)
            obs, reward, terminated, truncated, info = baseline_env.step(action)
            done = terminated or truncated
            total_reward += reward
        print(f"Baseline Episode {ep + 1}: reward = {total_reward}")
    baseline_env.close()

    # --- Trained model (After Training) ---
    print("\nRecording trained A2C agent episodes...")
    model = A2C.load(str(model_file))
    env = gym.make("FrozenLake-v1", is_slippery=True, render_mode="rgb_array")
    trained_env = RecordVideo(
        env,
        video_folder=str(video_dir),
        episode_trigger=lambda e: True,
        name_prefix="a2c_frozenlake_trained"
    )

    for ep in range(num_episodes):
        obs, info = trained_env.reset()
        done = False
        total_reward = 0
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            action = int(action)
            obs, reward, terminated, truncated, info = trained_env.step(action)
            done = terminated or truncated
            total_reward += reward
        print(f"Trained Episode {ep + 1}: reward = {total_reward}")
    trained_env.close()

record_demo_videos(model_file, video_dir)


Recording baseline A2C agent episodes...
Baseline Episode 1: reward = 0.0
Baseline Episode 2: reward = 0.0
Baseline Episode 3: reward = 0.0


/Users/davidjayakumar/Library/CloudStorage/OneDrive-AtlanticTU/YEAR 4/Project Engineering/Network-Defender/.venv_mac/lib/python3.9/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /Users/davidjayakumar/Library/CloudStorage/OneDrive-AtlanticTU/YEAR 4/Project Engineering/Network-Defender/documentation/frozenlake/a2c-frozenlake/run_2025-10-25_14-45-31/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Baseline Episode 4: reward = 0.0
Baseline Episode 5: reward = 0.0

Recording trained A2C agent episodes...
Trained Episode 1: reward = 1.0
Trained Episode 2: reward = 0.0
Trained Episode 3: reward = 1.0
Trained Episode 4: reward = 1.0
Trained Episode 5: reward = 1.0
